# Data Science Moringa Group Project - A Data Science Phase 3 Project
# Final Project Submission
#####:

Student name: Pete Njagi,
          

Student pace: DS-full time-Online

Scheduled project review date/time: March 10th 2024

###GitHub Link:
https://github.com/petezdj/

<img src="big%20data-1.jpg" alt="drawing" width="700"/>

#### Important Project Files:

1. index.ipynb (Main Juypter document)
2. Presentation.pdf (Presentation)
3. Data_sci_jobs.csv (Main Data)
4. Optional Project Pdf

## Introduction
KenyaData Insights, 